In [35]:
import pandas as pd
from census_api import census_df
from scrape_realtor import realtor_df

In [28]:
print(realtor_df.head())

  month_date_yyyymm postal_code            zip_name  median_listing_price  \
0            202208       99705      north pole, ak                363000   
1            202208       99615          kodiak, ak                341500   
2            202208       99829          hoonah, ak                200000   
3            202208       99515       anchorage, ak                452500   
4            202208       99737  delta junction, ak                375000   

   active_listing_count  median_days_on_market  new_listing_count  \
0                   113                     38                 44   
1                    36                    137                  0   
2                     1                     29                  0   
3                    27                     22                 20   
4                    41                     58                  8   

   price_increased_count  price_reduced_count  pending_listing_count  \
0                      0                   72     

In [40]:
print(census_api.census_df)


                County       State  Total:  Total:Hispanic or Latino  \
0       Autauga County     Alabama   58805                      2117   
1       Baldwin County     Alabama  231767                     12686   
2       Barbour County     Alabama   25223                      1510   
3          Bibb County     Alabama   22293                       740   
4        Blount County     Alabama   59134                      5771   
...                ...         ...     ...                       ...   
3216   Renville County   Minnesota   14723                      1319   
3217     Roseau County   Minnesota   15331                       238   
3218  Sherburne County   Minnesota   97183                      2820   
3219     Steele County   Minnesota   37406                      3140   
3220       Todd County   Minnesota   25262                      2208   

      Total:Not Hispanic or Latino:Population of one race:White alone  \
0                                                 41582       